In [4]:
import plotly.graph_objects as go
import os

In [5]:
def read_file(src):
    ch1_file = open(os.path.join(src))

    ch1 = ch1_file.read()

    ch1_file.close()
    
    ch1_lines = ch1.strip().split('\n')

    parsed_data = {}

    for i in range(23):
        key,value = ch1_lines[i].strip().split(',')[:2]
        parsed_data[key] = value
    
    volt = []

    for i in ch1_lines[25:]:
        try:
            volt.append(float(i.strip().split(',')[0]))        
        except:
            print('bad parsing data'+i)
        
    parsed_data['volt'] = volt
    return (parsed_data)

def get_shift(main,slave):
    if (len(main)!=[] and len(slave)!=[]):
        if (len(main)<2 or len(slave)<2):
            return -1
        elif (len(main)!=len(slave)):
            main = main[0:2]
            slave = slave[0:2]
    avg_shift = 0
    tmp = []
    for i in range(len(main)-1):
        tmp.append(abs(main[i+1]-main[i]))
        tmp.append(abs(slave[i+1]-slave[i]))

    avg_period = sum(tmp)/len(tmp)
    tmp = []
    for i in range(len(main)):
        tmp.append(slave[i]-main[i])

    avg_shift = sum(tmp)/len(tmp)

    return(avg_shift/avg_period*360)

def optimized_find_peaks(volt):
    peak_volt = max(volt)*0.8

    # This will storaged the result of the time where peak occured
    peak_position = []
    # TMP storage the index each group when peak value occurs
    # you can also see this as the time of each peak in group
    # since the time will be index * sample_period
    TMP = []
    # Flag used for determine the group 
    # When False and deteced peak voltage occurs 
    # it let program start to record a new group of peak point
    flag = False
    # target used to storage the first peak point in each peak 
    target = 0
    threshold = 10
    for i in range(len(volt)):
        if (volt[i] > peak_volt):
            target = i
            flag = True
            TMP.append(i)
        if (flag == True and (i>target+threshold or i==len(volt)-1)):
            if (len(TMP)>100):
                peak_position.append(sum(TMP)//len(TMP))
            TMP = [] 
            flag = False
    return(peak_position)

def fix(num):
    if num>150:
        return (num-360)
    if num<-150:
        return (num+360)
    else: 
        return num


# Read data and calculate phase

In [6]:
src_1 = './RLC_R_11.91'
src_2 = './RLC_R_993.9'
src_3 = './RLC_R_2200'

data_1 = []
for i in os.listdir(src_1):
    channels = {'ch1':'','ch2':''}
    channels['ch1'] = read_file(os.path.join(src_1,i,'1.CSV'))
    channels['ch2'] = read_file(os.path.join(src_1,i,'2.CSV'))
    data_1.append(channels)
volt_amp_1 = []
for i in range(len(data_1)):
    ch1_div = float(data_1[i]['ch1']['Vertical Scale'])
    ch2_div = float(data_1[i]['ch2']['Vertical Scale'])

    ch1_peaks = optimized_find_peaks(data_1[i]['ch1']['volt'])
    ch2_peaks= optimized_find_peaks(data_1[i]['ch2']['volt'])
    volt_amp_1.append(get_shift(ch1_peaks,ch2_peaks))

data_2 = []
for i in os.listdir(src_2):
    channels = {'ch1':'','ch2':''}
    channels['ch1'] = read_file(os.path.join(src_2,i,'1.CSV'))
    channels['ch2'] = read_file(os.path.join(src_2,i,'2.CSV'))
    data_2.append(channels)
volt_amp_2 = []
for i in range(len(data_2)):
    ch1_div = float(data_2[i]['ch1']['Vertical Scale'])
    ch2_div = float(data_2[i]['ch2']['Vertical Scale'])

    ch1_peaks = optimized_find_peaks(data_2[i]['ch1']['volt'])
    ch2_peaks= optimized_find_peaks(data_2[i]['ch2']['volt'])
    volt_amp_2.append(get_shift(ch1_peaks,ch2_peaks))

data_3 = []
for i in os.listdir(src_3):
    channels = {'ch1':'','ch2':''}
    channels['ch1'] = read_file(os.path.join(src_3,i,'1.CSV'))
    channels['ch2'] = read_file(os.path.join(src_3,i,'2.CSV'))
    data_3.append(channels)
volt_amp_3 = []
for i in range(len(data_3)):
    ch1_div = float(data_3[i]['ch1']['Vertical Scale'])
    ch2_div = float(data_3[i]['ch2']['Vertical Scale'])

    ch1_peaks = optimized_find_peaks(data_3[i]['ch1']['volt'])
    ch2_peaks = optimized_find_peaks(data_3[i]['ch2']['volt'])
    volt_amp_3.append(get_shift(ch1_peaks,ch2_peaks))


# Data pre-proccess & cleaning
clean up parse failed datapoint ( with value -1 which return by `optimized_find_peaks(volt)` )

In [7]:

volt_amp_1 = [fix(i) for i in volt_amp_1]
volt_amp_2 = [fix(i) for i in volt_amp_2]
volt_amp_3 = [fix(i) for i in volt_amp_3]

frequency_1 = [2000+i*200 for i in range(26) if volt_amp_1[i] != -1]
frequency_2 = [1000+i*200 for i in range(41) if volt_amp_2[i] != -1]
frequency_3 = [3000+i*2000 for i in range(33) if volt_amp_3[i] != -1]

volt_amp_1 = [fix(i) for i in volt_amp_1 if i != -1]
volt_amp_2 = [fix(i) for i in volt_amp_2 if i != -1]
volt_amp_3 = [fix(i) for i in volt_amp_3 if i != -1]

In [8]:
trace_1 = go.Scattergl(
    x=frequency_1,
    y=volt_amp_1,
    mode='markers',
    name= 'R = 11.91 , Q = 254.43 ' ,
)
trace_2 = go.Scattergl(
    x=frequency_2,
    y=volt_amp_2,
    mode='markers',
    name= 'R = 993.9 , Q = 3.05 ' ,
)
trace_3 = go.Scattergl(
    x=frequency_3,
    y=volt_amp_3,
    mode='markers',
    name= 'R = 2200 , Q = 1.37 ' ,
)

fig = go.Figure()
fig.add_trace(trace_1)
fig.add_trace(trace_2)
fig.add_trace(trace_3)


Xaxis_title = "Frequency"
Yaxis_title = "Phase Shift (degree) "
fig['layout'].update(xaxis_title=Xaxis_title,
                        yaxis_title=Yaxis_title,
                        legend_title='sources',
                        font=dict(family='Microsoft JhengHei', size=18,), xaxis=dict(tickangle=0)
                        )

fig.write_html('RLC_phase_shift.html', auto_open=True)